# Building an AI Agent with AGNO: Using Natural Language to Query SQLite Databases

## Introduction

AGNO is an open-source, lightweight library designed for building AI agents with memory, knowledge, tools, and reasoning capabilities. In this guide, we'll explore how to create an AI agent that can join tables and execute complex queries in a SQLite database using natural language instructions.

## Prerequisites

- Python 3.x (latest version)
- Required packages:
  ```
  agno
  sqlalchemy
  python-dotenv
  ```
- OpenAI API key in a .env file

## Setting Up the Database

Let's start by creating a sample SQLite database with customer and order data:

In [1]:
from sqlalchemy import create_engine, text

db_url = "sqlite:///example.db"
engine = create_engine(db_url)

with engine.connect() as connection:
    connection.execute(text("PRAGMA foreign_keys = ON;"))
    connection.execute(text("""
        CREATE TABLE IF NOT EXISTS customers (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            email TEXT
        );
    """))
    connection.execute(text("""
        CREATE TABLE IF NOT EXISTS orders (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            customer_id INTEGER,
            product TEXT,
            amount NUMERIC(10, 2),
            FOREIGN KEY (customer_id) REFERENCES customers(id)
        );
    """))

    # Insert sample data
    connection.execute(text("INSERT INTO customers (id, name, email) VALUES (1, 'John Doe', 'john@example.com');"))
    connection.execute(text("INSERT INTO customers (id, name, email) VALUES (2, 'Jane Smith', 'jane@example.com');"))
    connection.execute(text("INSERT INTO orders (id, customer_id, product, amount) VALUES (1, 1, 'Laptop', 999.99);"))
    connection.execute(text("INSERT INTO orders (id, customer_id, product, amount) VALUES (2, 1, 'Mouse', 19.99);"))
    connection.execute(text("INSERT INTO orders (id, customer_id, product, amount) VALUES (3, 2, 'Keyboard', 49.99);"))
    connection.commit()

This script creates two tables:
- **customers**: storing customer information
- **orders**: storing order details with a foreign key to customers

## Setting Up the API Key

First, create a .env file in your project directory:

```
OPENAI_API_KEY=your-openai-api-key-here
```

## Building the AGNO Agent

Now let's set up the AGNO agent that will interpret natural language and execute SQL queries:


In [2]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Verify API key is available
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found in .env file")

from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.sql import SQLTools

# Setup database connection
db_url = "sqlite:///example.db"

# Create the Agent with explicit API key
agent = Agent(
    model=OpenAIChat(
        "gpt-4o-mini", 
        api_key=openai_api_key  # Explicitly pass the API key
    ),
    tools=[SQLTools(db_url=db_url)],
    show_tool_calls=True,
    markdown=False
)

The key improvements in this code:
1. We explicitly load the .env file
2. We verify the API key exists before proceeding
3. We pass the API key directly to the OpenAIChat model

## Using the Agent for Natural Language Queries

Let's see how the agent handles different types of queries:

### Example 1: Retrieving joined data

In [3]:
response = agent.run("Get all customers and their orders")
print(response.content)

Here are the customers and their orders:

| Customer ID | Customer Name | Customer Email      | Order ID | Product | Amount  |
|-------------|---------------|---------------------|----------|---------|---------|
| 1           | John Doe     | john@example.com    | 1        | Laptop  | 999.99  |
| 1           | John Doe     | john@example.com    | 2        | Mouse   | 19.99   |
| 2           | Jane Smith   | jane@example.com    | 3        | Keyboard| 49.99   |

- **John Doe** has ordered a **Laptop** and a **Mouse**.
- **Jane Smith** has ordered a **Keyboard**.


The agent will generate and execute a query like:
```sql
SELECT * FROM customers JOIN orders ON customers.id = orders.customer_id;
```

### Example 2: Performing aggregations

In [4]:
response = agent.run("Get all customers and their total amount spent on their orders.")
print(response.content)

Here are the customers and their total amount spent on their orders:

| Customer ID | Customer Name | Total Spent |
|-------------|---------------|--------------|
| 1           | John Doe     | $1019.98    |
| 2           | Jane Smith   | $49.99      |


The agent will generate a more complex query:
```sql
SELECT c.name, SUM(o.amount) 
FROM customers c 
JOIN orders o ON c.id = o.customer_id 
GROUP BY c.name;
```

## Behind the Scenes

What makes this possible is AGNO's SQLTools integration. When you ask a question in natural language:

1. The model interprets your question
2. The SQLTools component translates it into a valid SQL query
3. The query executes against your database
4. Results are formatted and returned to you

AGNO handles all the complexity of query generation, database connections, and result formatting.

## Advantages of This Approach

- **Accessibility**: Non-technical users can query databases
- **Flexibility**: Works with complex SQL operations (joins, aggregations, filtering)
- **Efficiency**: Faster than writing SQL queries manually, especially for complex reports
- **Extensibility**: The same pattern works for other database engines (PostgreSQL, MySQL, etc.)

## Conclusion

With just a few lines of code, AGNO allows you to create an AI agent that can interpret natural language queries and execute appropriate SQL commands. This approach simplifies database interaction and can be extended to more complex database scenarios.

SQLite's lightweight nature makes it perfect for prototyping and small applications, while AGNO's flexibility allows it to work with various database systems.

## Next Steps

- Experiment with more complex queries
- Connect to larger databases
- Add memory capabilities to your agent
- Combine with other AGNO tools for more capabilities

Ready to build your own natural language database interface? Give AGNO a try and see how it can enhance your applications!